# Finding Similarities in Manhattan and Toronto Neighborhoods

Import required libraries:

In [1]:
import pandas as pd
import numpy as np
import json
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
from geopy.geocoders import Nominatim
import requests

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## 1. Data Acquisition

### Toronto Neighborhoods

Taken from week 3 in condensed form. First we obtain neighborhood data from a wikipedia page:

In [2]:
# Use older version of wikipedia page, works better for pandas
postal_codes = pd.read_html("https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1012118802", header=0)[0]
postal_codes.rename(columns={'Postal code': 'Postal Code', 'Neighbourhood': 'Neighborhood'}, inplace = True)
postal_codes_filtered = postal_codes[postal_codes['Borough'] != 'Not assigned']
postal_codes_grouped = postal_codes_filtered.groupby(['Postal Code', 'Borough'])['Neighborhood'].apply(','.join).reset_index()
postal_codes_grouped.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Then we add geospatial data:

In [4]:
lat_long = pd.read_csv('Geospatial_Coordinates.csv')
toronto_data = pd.merge(postal_codes_grouped, lat_long, on='Postal Code')
toronto_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Manhattan Neighborhoods

Taken also from week 3

In [8]:
!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json
with open('newyork_data.json') as json_data:
    newyork_json_data = json.load(json_data)

column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
newyork_data = pd.DataFrame(columns=column_names)
for data in newyork_json_data['features']:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    newyork_data = newyork_data.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
manhattan_data = newyork_data[newyork_data['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


### Venues in Manhattan and Toronto

In [9]:
CLIENT_ID = '#' # censored
CLIENT_SECRET = '#' # censored
VERSION = '20180605'
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

In [10]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    print('Querying started')
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    print('Querying finished')
    return(nearby_venues)

In [11]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Querying started
Querying finished


In [12]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Querying started
Querying finished


Check the data Foursquare returned for both cities:

In [13]:
print(toronto_venues.shape)
print(manhattan_venues.shape)
print('There are {} uniques categories for Toronto and {} unique categories for Manhattan.'.format(len(manhattan_venues['Venue Category'].unique()),len(toronto_venues['Venue Category'].unique())))
toronto_venues.head()

(2136, 7)
(3259, 7)
There are 336 uniques categories for Toronto and 272 unique categories for Manhattan.


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Sail Sushi,43.765951,-79.191275,Restaurant


Check how many venue categories they have in common:

In [54]:
common_venue_categories = pd.merge(toronto_venues['Venue Category'], manhattan_venues['Venue Category'], on='Venue Category')
common_venue_categories['Venue Category'].unique().shape

(211,)

## 2. Data Wrangling

In [14]:
toronto_append = toronto_data
toronto_append['City'] = 'T'
manhattan_append = manhattan_data
manhattan_append['City'] = 'M'
combined_data = toronto_append.append(manhattan_append, ignore_index=True)
cols = list(combined_data.columns)
cols.remove('City')
fixed_columns = ['City'] + cols
combined_venues = combined_data[fixed_columns]
combined_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,City
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,T
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,T
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,T
3,M1G,Scarborough,Woburn,43.770992,-79.216917,T
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,T


Next we one-hot encode the data, after combining Toronto and Manhattan venues

In [15]:
toronto_append = toronto_venues
toronto_append['City'] = 'T'
manhattan_append = manhattan_venues
manhattan_append['City'] = 'M'
combined_venues = toronto_append.append(manhattan_append, ignore_index=True)
cols = list(combined_venues.columns)
cols.remove('City')
fixed_columns = ['City'] + cols
combined_venues = combined_venues[fixed_columns]
combined_venues.shape

(5395, 8)

In [16]:
combined_onehot = pd.get_dummies(combined_venues[['Venue Category']], prefix="", prefix_sep="")
combined_onehot['Neighborhood'] = combined_venues['Neighborhood']
combined_onehot['City'] = combined_venues['City']
# Move column neighorhoods back to first position
cols = list(combined_onehot.columns)
cols.remove('Neighborhood')
cols.remove('City')
fixed_columns = ['Neighborhood', 'City'] + cols
combined_onehot = combined_onehot[fixed_columns]
combined_grouped = combined_onehot.groupby(['Neighborhood', 'City']).mean().reset_index()
combined_grouped.head()

,Neighborhood,City,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Auditorium,College Bookstore,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Curling Ice,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Dentist's Office,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dive Shop,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean BBQ Restaurant,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors 

Make combined data human-readable by displaying top 10 venues

In [17]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[2:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [36]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood', 'City']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
combined_venues_sorted = pd.DataFrame(columns=columns)
combined_venues_sorted['Neighborhood'] = combined_grouped['Neighborhood']
combined_venues_sorted['City'] = combined_grouped['City']

for ind in np.arange(combined_grouped.shape[0]):
    combined_venues_sorted.iloc[ind, 2:] = return_most_common_venues(combined_grouped.iloc[ind, :], num_top_venues)

combined_venues_sorted.head()

,Neighborhood,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,T,Latin American Restaurant,Clothing Store,Skating Rink,Lounge,Breakfast Spot,Cuban Restaurant,Ethiopian Restaurant,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant
1,"Alderwood, Long Branch",T,Pizza Place,Pub,Gym,Athletics & Sports,Coffee Shop,Skating Rink,Sandwich Place,Farm,Eastern European Restaurant,Dog Run
2,"Bathurst Manor, Wilson Heights, Downsview North",T,Bank,Coffee Shop,Middle Eastern Restaurant,Deli / Bodega,Supermarket,Shopping Mall,Sushi Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Gas Station
3,Battery Park City,M,Park,Coffee Shop,Clothing Store,Hotel,Gym,Memorial Site,Playground,Women's Store,Pizza Place,Beer Garden
4,Bayview Village,T,Bank,Café,Japanese Restaurant,Chinese Restaurant,Yoga Studio,Ethiopian Restaurant,Drugstore,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant


## 3. Clustering

We use k-Means clustering to cluster the combined Manhattan and Toronto data

In [37]:
# set number of clusters
kclusters = 5

combined_clustering = combined_grouped.drop(columns=['Neighborhood', 'City'])

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(combined_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_.shape

(135,)

In [38]:
# add clustering labels
combined_venues_clustered = combined_venues_sorted
combined_venues_clustered.insert(0, 'Cluster Labels', kmeans.labels_)
combined_venues_clustered.tail()

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
combined_venues_clustered_merged = combined_data.join(combined_venues_clustered.set_index(['Neighborhood', 'City']), on=['Neighborhood', 'City'], how='right')

# remove neighborhoods for which no venue data could be found
combined_venues_clustered_merged = combined_venues_clustered_merged[~combined_venues_clustered_merged['Cluster Labels'].isnull()]

combined_venues_clustered_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,T,3,Fast Food Restaurant,Yoga Studio,Ethiopian Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,T,2,Bar,Yoga Studio,Event Space,Drugstore,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,T,1,Donut Shop,Restaurant,Breakfast Spot,Mexican Restaurant,Electronics Store,Intersection,Medical Center,Rental Car Location,Bank,Drugstore
3,M1G,Scarborough,Woburn,43.770992,-79.216917,T,1,Coffee Shop,Korean BBQ Restaurant,Mexican Restaurant,Yoga Studio,Ethiopian Restaurant,Drugstore,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,T,1,Gas Station,Fried Chicken Joint,Bank,Caribbean Restaurant,Hakka Restaurant,Athletics & Sports,Bakery,Thai Restaurant,Farmers Market,Farm


We visualize the clustering result with colored dots over a Manhattan and Toronto map with folium

In [39]:
def createMap(lat, long, data):
    # create map
    map_clusters = folium.Map(location=[lat, long], zoom_start=11)

    # set color scheme for the clusters
    x = np.arange(kclusters)
    ys = [i + x + (i*x)**2 for i in range(kclusters)]
    colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
    rainbow = [colors.rgb2hex(i) for i in colors_array]

    # add markers to the map
    markers_colors = []
    for lat, lon, poi, cluster in zip(data['Latitude'], data['Longitude'], data['Neighborhood'], data['Cluster Labels']):
        label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
        cluster = int(cluster)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[cluster-1],
            fill=True,
            fill_color=rainbow[cluster-1],
            fill_opacity=0.7).add_to(map_clusters)

    return map_clusters

In [40]:
createMap(43.70011, -79.4163, combined_venues_clustered_merged[combined_venues_clustered_merged['City'] == 'T'])

In [41]:
createMap(40.78343, -73.96625, combined_venues_clustered_merged[combined_venues_clustered_merged['City'] == 'M'])

### Examine clusters and interpret the results

First cluster - Toronto neighborhoods with parks, shops and exotic restaurants

In [42]:
combined_venues_clustered_merged.loc[combined_venues_clustered_merged['Cluster Labels'] == 0, combined_venues_clustered_merged.columns[[1,2] + list(range(5, combined_venues_clustered_merged.shape[1]))]]

,Borough,Neighborhood,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Scarborough,"Milliken, Agincourt North, Steeles East, L'Amo...",T,0,Playground,Park,Intersection,Yoga Studio,Ethiopian Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant
21,North York,"Willowdale, Newtonbrook",T,0,Park,Yoga Studio,Escape Room,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store
23,North York,York Mills West,T,0,Convenience Store,Park,Escape Room,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store
40,East York,"East Toronto, Broadview North (Old East York)",T,0,Park,Convenience Store,Intersection,Ethiopian Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store
50,Downtown Toronto,Rosedale,T,0,Park,Playground,Trail,Yoga Studio,English Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Duty-free Shop
74,York,Caledonia-Fairbanks,T,0,Park,Pool,Women's Store,Afghan Restaurant,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant
98,York,Weston,T,0,Convenience Store,Park,Escape Room,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store


Second cluster - Rest

In [43]:
combined_venues_clustered_merged.loc[combined_venues_clustered_merged['Cluster Labels'] == 1, combined_venues_clustered_merged.columns[[1,2] + list(range(5, combined_venues_clustered_merged.shape[1]))]]

,Borough,Neighborhood,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,"Guildwood, Morningside, West Hill",T,1,Donut Shop,Restaurant,Breakfast Spot,Mexican Restaurant,Electronics Store,Intersection,Medical Center,Rental Car Location,Bank,Drugstore
3,Scarborough,Woburn,T,1,Coffee Shop,Korean BBQ Restaurant,Mexican Restaurant,Yoga Studio,Ethiopian Restaurant,Drugstore,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store
4,Scarborough,Cedarbrae,T,1,Gas Station,Fried Chicken Joint,Bank,Caribbean Restaurant,Hakka Restaurant,Athletics & Sports,Bakery,Thai Restaurant,Farmers Market,Farm
6,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",T,1,Convenience Store,Hobby Shop,Discount Store,Coffee Shop,Department Store,Bus Station,Chinese Restaurant,Farmers Market,Farm,Drugstore
9,Scarborough,"Birch Cliff, Cliffside West",T,1,Café,Farm,College Stadium,General Entertainment,Skating Rink,Yoga Studio,English Restaurant,Drugstore,Dumpling Restaurant,Duty-free Shop
10,Scarborough,"Dorset Park, Wexford Heights, Scarborough Town...",T,1,Indian Restaurant,Chinese Restaurant,Thrift / Vintage Store,Light Rail Station,Vietnamese Restaurant,Pet Store,English Restaurant,Doner Restaurant,Donut Shop,Drugstore
11,Scarborough,"Wexford, Maryvale",T,1,Smoke Shop,Vietnamese Restaurant,Bakery,Auto Garage,Sandwich Place,Middle Eastern Restaurant,Filipino Restaurant,English Restaurant,Fish Market,Donut Shop
12,Scarborough,Agincourt,T,1,Latin American Restaurant,Clothing Store,Skating Rink,Lounge,Breakfast Spot,Cuban Restaurant,Ethiopian Restaurant,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant
13,Scarborough,"Clarks Corners, Tam O'Shanter, Sullivan",T,1,Pizza Place,Fast Food Restaurant,Thai Restaurant,Intersection,Bank,Italian Restaurant,Fried Chicken Joint,Noodle House,Pharmacy,Chinese Restaurant
15,Scarborough,"Steeles West, L'Amoreaux West",T,1,Fast Food Restaurant,Pizza Place,Pharmacy,Bank,Chinese Restaurant,Coffee Shop,Intersection,Sandwich Place,Breakfast Spot,Cupcake Shop


Third cluster - Outlier

In [44]:
combined_venues_clustered_merged.loc[combined_venues_clustered_merged['Cluster Labels'] == 2, combined_venues_clustered_merged.columns[[1,2] + list(range(5, combined_venues_clustered_merged.shape[1]))]]

,Borough,Neighborhood,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,"Rouge Hill, Port Union, Highland Creek",T,2,Bar,Yoga Studio,Event Space,Drugstore,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant


Fourth cluster - Outlier

In [45]:
combined_venues_clustered_merged.loc[combined_venues_clustered_merged['Cluster Labels'] == 3, combined_venues_clustered_merged.columns[[1,2] + list(range(5, combined_venues_clustered_merged.shape[1]))]]

,Borough,Neighborhood,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,"Malvern, Rouge",T,3,Fast Food Restaurant,Yoga Studio,Ethiopian Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant


Fifth cluster - Neighborhoods with recreational venues

In [46]:
combined_venues_clustered_merged.loc[combined_venues_clustered_merged['Cluster Labels'] == 4, combined_venues_clustered_merged.columns[[1,2] + list(range(5, combined_venues_clustered_merged.shape[1]))]]

,Borough,Neighborhood,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,Scarborough Village,T,4,Playground,Jewelry Store,Yoga Studio,Ethiopian Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store
7,Scarborough,"Golden Mile, Clairlea, Oakridge",T,4,Bakery,Ice Cream Shop,Metro Station,Park,Intersection,Bus Line,Soccer Field,Yoga Studio,Escape Room,Drugstore
8,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",T,4,Motel,American Restaurant,Yoga Studio,Event Space,Drugstore,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant
17,North York,Hillcrest Village,T,4,Pool,Golf Course,Fast Food Restaurant,Mediterranean Restaurant,Dog Run,Creperie,Coworking Space,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant
25,North York,Parkwoods,T,4,Food & Drink Shop,Park,Fast Food Restaurant,Escape Room,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant
30,North York,Downsview,T,4,Grocery Store,Park,Playground,Business Service,Bank,Liquor Store,Gym / Fitness Center,Discount Store,Athletics & Sports,Shopping Mall
31,North York,Downsview,T,4,Grocery Store,Park,Playground,Business Service,Bank,Liquor Store,Gym / Fitness Center,Discount Store,Athletics & Sports,Shopping Mall
32,North York,Downsview,T,4,Grocery Store,Park,Playground,Business Service,Bank,Liquor Store,Gym / Fitness Center,Discount Store,Athletics & Sports,Shopping Mall
33,North York,Downsview,T,4,Grocery Store,Park,Playground,Business Service,Bank,Liquor Store,Gym / Fitness Center,Discount Store,Athletics & Sports,Shopping Mall
36,East York,Woodbine Heights,T,4,Spa,Curling Ice,Athletics & Sports,Park,Beer Store,Skating Rink,Yoga Studio,Escape Room,Drugstore,Dumpling Restaurant
